# Analyse du nombre d'opérations par pixel

Ce notebook propose une analyse mathématique du nombre d'opérations (multiply‑add) par pixel  pour deux réseaux de débruitage populaires : **FFDNet** et **DRUNet**.     Nous utilisons les implémentations officielles (IPOL pour FFDNet, DeepInv pour DRUNet) afin de charger les modèles, puis nous comparons :

- une **estimation analytique**, basée sur les paramètres exacts du modèle chargé (nombre de couches, canaux, tailles de noyaux et facteurs de sous‑échantillonnage/sur‑échantillonnage) ;
- une **estimation numérique** en évaluant les FLOPs totaux via `ptflops` et en normalisant par le nombre de pixels de l'entrée.

Afin d'éviter toute ambiguïté sur l'entrée du modèle, nous enveloppons les réseaux pour y fixer explicitement le paramètre de bruit (`noise_sigma` ).

## 1. Chargement de FFDNet et analyse

Nous téléchargeons la version IPOL de FFDNet, importons dynamiquement `models.py`,     instancions le modèle couleur (3 canaux) et chargeons ses poids pré‑entraînés.

In [1]:
import os, sys, zipfile, urllib.request, torch

URL_CODE = "https://www.ipol.im/pub/art/2019/231/ffdnet-pytorch39.zip"
ZIP_PATH, EXTRACT_DIR = "ffdnet-pytorch39.zip", "ffdnet_ipol"

# 1) Téléchargement + extraction 
if not os.path.exists(EXTRACT_DIR):
    if not os.path.exists(ZIP_PATH):
        print("Téléchargement FFDNet (IPOL)…")
        urllib.request.urlretrieve(URL_CODE, ZIP_PATH)
    with zipfile.ZipFile(ZIP_PATH, "r") as z:
        z.extractall(EXTRACT_DIR)
    print(f"Extrait dans {EXTRACT_DIR}/")

# 2) Ajouter le dossier aux paths
FFDNET_DIR = os.path.join(EXTRACT_DIR, "ffdnet-pytorch")
sys.path.append(FFDNET_DIR)
from models import FFDNet  

# 3) Localiser les poids dans le dossier extrait
candidates = [
    os.path.join(FFDNET_DIR, "models", "net_rgb.pth"),
    os.path.join(FFDNET_DIR, "net_rgb.pth"),
]

WEIGHTS_PATH = next((p for p in candidates if os.path.exists(p)), None)
assert WEIGHTS_PATH, "Poids 'net_rgb.pth' introuvables dans l'archive extraite."

# 4) Charger le modèle + poids
ffdnet = FFDNet(num_input_channels=3)
state = torch.load(WEIGHTS_PATH, map_location="cpu")
ffdnet.load_state_dict(state.get("state_dict", state), strict=False)
ffdnet.eval()
print("FFDNet (RGB) chargé et prêt")

#


Téléchargement FFDNet (IPOL)…
Extrait dans ffdnet_ipol/
FFDNet (RGB) chargé et prêt


### 1.1 Estimation analytique pour FFDNet

FFDNet commence par un **sous-échantillonnage ×2** en hauteur et en largeur, produisant 4 sous-images.  
Toutes les convolutions 3×3 opèrent donc sur cette résolution réduite d’un facteur 4.  
Pour revenir au coût par pixel de l’image d’origine, on divise systématiquement par 4.

On note :
- $C$ : nombre de canaux de l’image (3 pour la couleur, 1 pour le gris),
- $s$ : nombre de cartes de bruit concaténées à l’entrée,
- $W$ : largeur interne (nombre de feature maps de la première convolution),
- $D$ : nombre total de convolutions dans le réseau.

La formule analytique du coût par pixel est :

$$
\mathrm{MACs/pixel} \;=\;\tfrac{9}{4}\Big( W(4C+s) \;+\; (D-2)W^2 \;+\; 4CW \Big).
$$

- Le premier terme $W(4C+s)$ correspond à la **première convolution** (entrée → largeur interne).  
- Le terme $(D-2)W^2$ regroupe les **convolutions intermédiaires** de largeur constante $W$.  
- Le dernier terme $4CW$ correspond à la **dernière convolution** (largeur interne → sortie avec $4C$ canaux).  
- Le facteur $9$ provient du noyau $3\times3$, et la division par 4 compense la résolution réduite.



In [62]:
import torch

def ffdnet_macs_per_pixel(model, C=3, s=1):
    """
    Calcul du coût analytique en MACs/pixel pour FFDNet

        MACs/pixel = (9/4) [ W(4C+s) + (D-2)W^2 + 4CW ]

    Paramètres
    ----------
    model : nn.Module
        Instance FFDNet.
    C : int
        Nombre de canaux de l'image (3 = couleur, 1 = gris).
    s : int
        Nombre de cartes sigma (1 par canal).
    """
    # Nombre total de convolutions
    convs = [m for m in model.modules() if isinstance(m, torch.nn.Conv2d)]
    D = len(convs)

    # Largeur interne = nombre de cartes de la première conv (out_channels)
    W = convs[0].out_channels

    # Application de la formule analytique
    macs_per_pixel = (9/4) * (W * (4*C + s) + (D - 2) * W * W + 4*C * W)
    return macs_per_pixel


analytic_ffdnet = ffdnet_macs_per_pixel(ffdnet, C=3, s=3)
print("MACs/pixel :", analytic_ffdnet)
print("Opération/pixel :", analytic_ffdnet * 2)


MACs/pixel : 213192.0
Opération/pixel : 426384.0


### 1.2 Estimation numérique via ptflops

L'implémentation FFDNet IPOL s'utilise comme `out = model(x, noise_sigma)` où `noise_sigma` est un scalaire par image.     Afin d'éviter les erreurs lors de l'appel par `ptflops`, on encapsule le modèle dans un wrapper     qui injecte automatiquement `noise_sigma` et expose un `forward(x)` classique.

In [63]:
from ptflops import get_model_complexity_info
import torch.nn as nn

class FFDNetWrapper(nn.Module):
    def __init__(self, model, sigma=25.0/255.0):
        super().__init__()
        self.model = model
        self.sigma = sigma
    def forward(self, x):
        # FFDNet IPOL attend un tenseur 1D de taille (batch,) pour noise_sigma
        noise_sigma = torch.tensor([self.sigma], device=x.device)
        return self.model(x, noise_sigma)

ffdnet_wrap = FFDNetWrapper(ffdnet, sigma=25.0/255.0)

# Mesure des Opérations totaux avec ptflops
macs_ffdnet, _ = get_model_complexity_info(
    ffdnet_wrap, (3, 256, 256), as_strings=False, print_per_layer_stat=False
)
print("MACs/pixel (numérique) :", macs_ffdnet / (256 * 256))
print("Opérations/pixel (numérique) :", macs_ffdnet * 2 / (256 * 256))


MACs/pixel (numérique) : 214200.0
Opérations/pixel (numérique) : 428400.0


## 2. Chargement de DRUNet et analyse

Nous utilisons la librairie `deepinv` pour télécharger et instancier DRUNet pré‑entraîné.

In [30]:

import deepinv as dinv

drunet = dinv.models.DRUNet(pretrained="download").eval()
print("DRUNet chargé")


DRUNet chargé


### 2.1 Estimation analytique pour DRUNet

DRUNet suit une architecture de type **U-Net résiduel**, avec une tête de convolution, un encodeur multi-niveaux, un bottleneck, puis un décodeur symétrique.  
On peut récupérer ses hyperparamètres directement depuis le modèle :
- `nc` : liste du nombre de canaux à chaque niveau (ex. [64, 128, 256, 512]),
- `nb` : nombre de blocs résiduels par niveau (chaque bloc = 2 convolutions 3×3),
- `in_nc` : nombre de canaux d’entrée (en général 3),
- un canal supplémentaire est ajouté pour la carte de bruit.

Chaque convolution est comptée en multiplications-accumulations (MACs), puis normalisée par pixel d’entrée grâce aux facteurs de résolution :

$$
r_\ell = \frac{1}{4^\ell}, \quad \ell=0,\dots,L,
$$

avec $L=3$ pour un encodeur à 3 niveaux (résolutions relatives : 1, 1/4, 1/16, 1/64).

La formule est donc :

$$
\begin{aligned}
\mathrm{MACs/pixel} \;=\;& \underbrace{9\,r_0\,(in_{c}+1)\,nc_0}_{\text{tête}} \\
&+ \sum_{i=0}^{L-1} \Big[ \underbrace{18\,r_i\,nb\,nc_i^2}_{\text{blocs enc}} + \underbrace{4\,r_{i+1}\,nc_i\,nc_{i+1}}_{\text{down $2\times2$}} \Big] \\
&+ \underbrace{18\,r_L\,nb\,nc_L^2}_{\text{bottleneck}} \\
&+ \sum_{i=L-1}^{0} \Big[ \underbrace{4\,r_i\,nc_{i+1}\,nc_i}_{\text{up $2\times2$}} + \underbrace{18\,r_i\,nb\,nc_i^2}_{\text{blocs dec}} \Big] \\
&+ \underbrace{9\,r_0\,nc_0\,in_c}_{\text{queue}} .
\end{aligned}
$$


- La **tête** projette $(in\_nc+1)$ canaux (image + carte de bruit) vers $nc_0$.  
- Chaque **bloc résiduel** contient deux convolutions 3×3 → facteur $18\,r_i\,nc_i^2$.  
- Les **downsamplings** sont modélisés par des convolutions 2×2 stride 2.  
- Les **upsamplings** par des transposed-conv 2×2 stride 2.  
- La **queue** ramène $nc_0 \to in\_nc$.  
- Aucun terme de “fusion” séparée n’est ajouté : la concaténation des skip connections est absorbée dans le premier bloc du décodeur.



In [65]:
def drunet_macs_per_pixel(model):
    # Paramètres du modèle (avec valeurs par défaut)
    nc = model.nc if hasattr(model, 'nc') else [64, 128, 256, 512]
    nb = model.nb if hasattr(model, 'nb') else 4
    in_nc = model.in_nc if hasattr(model, 'in_nc') else 3

    L = len(nc) - 1  # nombre de transitions (niveaux enc/dec)
    res_factors = [1.0 / (4 ** i) for i in range(L + 1)]  # r_i = 1/4^i

    ops = 0.0

    # Tête (3x3): (C + sigma) -> n0, comptée à r0
    ops += (in_nc + 1) * nc[0] * 9 * res_factors[0]

    # Encodeur: blocs + down
    for i in range(L):
        ch = nc[i]
        # nb blocs résiduels (2 conv 3x3) à r_i
        ops += nb * 2 * ch * ch * 9 * res_factors[i]
        # down: conv 2x2 stride 2, coût à r_{i+1}
        ops += ch * nc[i + 1] * 4 * res_factors[i + 1]

    # Bottleneck au niveau L
    ch_body = nc[-1]
    ops += nb * 2 * ch_body * ch_body * 9 * res_factors[L]

    # Décodeur: up + blocs (sans fusion séparée)
    for i in reversed(range(L)):
        ch_in = nc[i + 1]
        ch_out = nc[i]
        # up: deconv 2x2 stride 2, coût à r_i
        ops += ch_in * ch_out * 4 * res_factors[i]
        # blocs résiduels (2 conv 3x3) à r_i
        ops += nb * 2 * ch_out * ch_out * 9 * res_factors[i]

    # Queue (3x3): n0 -> C à r0
    ops += nc[0] * in_nc * 9 * res_factors[0]

    return ops

analytic_drunet = drunet_macs_per_pixel(drunet)
print("MACs/pixel (analytique, DRUNet) :", analytic_drunet)
print("FLOPs/pixel (analytique) :", analytic_drunet * 2)


MACs/pixel (analytique, DRUNet) : 2191296.0
FLOPs/pixel (analytique) : 4382592.0


### 2.2 Estimation numérique via ptflops pour DRUNet

L'implémentation DeepInv de DRUNet utilise la signature `forward(x, sigma)`.     Pour éviter l'erreur de paramètre manquant, on enveloppe également DRUNet dans un wrapper qui fixe `sigma`.

In [53]:

from ptflops import get_model_complexity_info
import torch.nn as nn

class DRUNetWrapper(nn.Module):
    def __init__(self, model, sigma=25.0/255.0):
        super().__init__()
        self.model = model
        self.sigma = sigma
    def forward(self, x):
        # sigma est un scalaire passé à drunet
        return self.model(x, sigma=self.sigma)

# Envelopper le modèle
wrapped_drunet = DRUNetWrapper(drunet, sigma=25.0/255.0)

macs_drunet, _ = get_model_complexity_info(
    wrapped_drunet, (3, 256, 256), as_strings=False, print_per_layer_stat=False
)
print("MACs/pixel (numérique) :", macs_drunet / (256 * 256))
print("FLOPs/pixel (numérique) :", (macs_drunet * 2) / (256 * 256))


MACs/pixel (numérique) : 2119424.0
FLOPs/pixel (numérique) : 4238848.0


## 3. Synthèse des résultats

Le fonction ci‑dessous compare les estimations analytiques et numériques pour les deux modèles.

In [58]:
import numpy as np

def compare_macs_model(model_name, macs_analytic, macs_numeric):
    """
    Compare deux estimations de MACs/pixel (analytique vs numérique)
    pour un modèle donné (FFDNet, DRUNet, ...).

    Paramètres
    ----------
    model_name : str
        Nom du modèle (ex: "FFDNet", "DRUNet").
    macs_analytic : float
        Valeur analytique (MACs/pixel).
    macs_numeric : float
        Valeur numérique (ptflops, MACs/pixel).
    """
    diff_abs = np.abs(macs_numeric - macs_analytic)
    diff_rel = 100 * diff_abs / macs_analytic if macs_analytic != 0 else float("nan")

    print(f"=== Comparaison MACs/pixel : {model_name} ===")
    print(39 * "-")
    print(f"Analytique (MACs/pixel) : {macs_analytic:.2f}")
    print(f"Numérique (MACs/pixel)  : {macs_numeric:.2f}")
    print(f"Écart absolu            : {diff_abs:.2f}")
    print(f"Écart relatif (%)       : {diff_rel:.3f}%\n")

    return

compare_macs_model("FFDNet", analytic_ffdnet, macs_ffdnet / (256 * 256))
compare_macs_model("DRUNet", analytic_drunet, macs_drunet / (256 * 256))


=== Comparaison MACs/pixel : FFDNet ===
---------------------------------------
Analytique (MACs/pixel) : 213192.00
Numérique (MACs/pixel)  : 214200.00
Écart absolu            : 1008.00
Écart relatif (%)       : 0.473%

=== Comparaison MACs/pixel : DRUNet ===
---------------------------------------
Analytique (MACs/pixel) : 2191296.00
Numérique (MACs/pixel)  : 2119424.00
Écart absolu            : 71872.00
Écart relatif (%)       : 3.280%

